# Exercise: Compute summary statistics for the patient data

In [21]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd

# Set some Pandas options: maximum number of rows/columns it's going to display
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.max_columns', 10)

# Load the patient data

In [22]:
df = pd.read_csv('processed_data_predimed.csv')

In [23]:
df.shape

(6245, 17)

In [24]:
df.head()

,patient-id,location-id,sex,age,smoke,bmi,waist,wth,htn,diab,hyperchol,famhist,hormo,p14,toevent,event,group
0,1,1,Female,77,Never,25.92,94,0.657343,Yes,No,Yes,Yes,No,9,5.538672,No,MedDiet + VOO
1,2,1,Female,68,Never,34.85,150,0.949367,Yes,No,Yes,Yes,NaN,10,3.063655,No,MedDiet + Nuts
2,3,1,Female,66,Never,37.50,120,0.750000,Yes,Yes,No,No,No,6,5.590691,No,MedDiet + Nuts
3,4,1,Female,77,Never,29.26,93,0.628378,Yes,Yes,No,No,No,6,5.456537,No,MedDiet + VOO
4,5,1,Female,60,Never,30.02,104,0.662420,Yes,No,Yes,No,No,9,2.746064,No,Control


# 1. Did the mediterrean diet help prevent cardiovascular events?

To answer this question, we need to compute how many cardiovascular "events" occured in each group of participants, separated by the diet they followed.
In the data the column `event` contains `Yes` or `No`, indicating if that patient had an cardiovascular event. The column `group` contains which diet they followed.

* Convert the column `event` from string to binary (1 for Yes, 0 for No) (this will ease the calculations that follow later).
  Hint: use the method `.map()`

In [25]:
df['event'] = df['event'].map({'Yes': 1, 'No': 0})

* Now compute the total number of events by diet group. Compare the numbers and see if you can answer the question.

In [42]:
df.groupby('group')['event'].sum()

group
Control           96
MedDiet + Nuts    69
MedDiet + VOO     83
Name: event, dtype: int64

* Check how many patients had each group

In [44]:
df.groupby('group')['event'].count()

group
Control           2016
MedDiet + Nuts    2077
MedDiet + VOO     2152
Name: event, dtype: int64

There were no equal number in each group, so to be precise we need to put the numbers into perspective of the total. For that:
* Calculate how many events occured relative to the amount of patients in each group (in percentage). 
Do this sepearated by diet group.

In [46]:
df.groupby('group')['event'].sum()*100 / df.groupby('group')['event'].count()

group
Control           4.761905
MedDiet + Nuts    3.322099
MedDiet + VOO     3.856877
Name: event, dtype: float64

It seems that the control group had a higher percentage of events than the other two

# 2. Smoking

Did smoking make a difference in the outcome of the study?
Calculate how many events occured by diet group *and* smoking. The idea is that you arrive to a table like this:

| group          |   Current |   Former |   Never |
|:---------------|----------:|---------:|--------:|
| Control        |     ...   |    ...   |   ...   |
| MedDiet + Nuts |     ...   |    ...   |   ...   |
| MedDiet + VOO  |     ...   |    ...   |   ...   |

where each entry in the table has the percentage of events for each group
Hint: use `pivot_table`

In [50]:
counts = df.pivot_table(index='group', columns='smoke', values='event', aggfunc='sum')
counts

smoke,Current,Former,Never
group,,,
Control,13,39,44
MedDiet + Nuts,15,20,34
MedDiet + VOO,20,29,34


In [51]:
N = df.pivot_table(index='group', columns='smoke', values='event', aggfunc='count')
N

smoke,Current,Former,Never
group,,,
Control,264,485,1267
MedDiet + Nuts,291,539,1247
MedDiet + VOO,290,531,1331


In [52]:
counts/N

smoke,Current,Former,Never
group,,,
Control,0.049242,0.080412,0.034728
MedDiet + Nuts,0.051546,0.037106,0.027265
MedDiet + VOO,0.068966,0.054614,0.025545


# 3. Age differences?

Finally, check that there were no big differences in the age between the groups.
* Calculate the mean and standard deviation of the patient's age, separated by diet group.

You should be getting a table like this:

| group          |   Female |   Male |
|:---------------|---------:|-------:|
| Control        |     68   |   66.4 |
| MedDiet + Nuts |     67.4 |   65.8 |
| MedDiet + VOO  |     67.7 |   66.1 |


In [78]:
# this works but it is longer than necessary
df.groupby(['group', 'sex'])['age'].mean().reset_index().pivot_table(index='group', columns='sex', values='age').round(1)

sex,Female,Male
group,,
Control,68.0,66.4
MedDiet + Nuts,67.4,65.8
MedDiet + VOO,67.7,66.1


In [79]:
# instead of grouping first, do the pivot first and pass the aggregation function as an argument
df.pivot_table(index='group', columns='sex', values='age', aggfunc='mean').round(1)

sex,Female,Male
group,,
Control,68.0,66.4
MedDiet + Nuts,67.4,65.8
MedDiet + VOO,67.7,66.1


In [80]:
# you can have more than one aggregation function
df.pivot_table(index='group', columns='sex', values='age', aggfunc=['mean', 'std']).round(1)

mean          std     
sex            Female  Male Female Male
group                                  
Control          68.0  66.4    6.0  6.6
MedDiet + Nuts   67.4  65.8    5.6  6.4
MedDiet + VOO    67.7  66.1    5.8  6.6